# Serverless

### Introduction

In this lesson, we'll get started with infrastructure as code with the [serverless](https://www.serverless.com) framework.  Serverless is an infrastructure as code tool, meaning that it will allow us to create all of the resources that we need for our data pipeline, just by using some code.

In this lesson, we'll get started using serverless, and move through some of the benefits of infrastructure as code.

### Setting up serverless

Serverless is a node library (node is a backend javascript library).  So to get started with serverless, we first need to node install node.

On a mac, we can do so using homebrew:

```bash
brew install node
```

And then from there, we can install the serverless library.

```bash
npm install -g serverless
```

You can check that it worked by just typing `serverless` into the command line.

```bash
serverless
```

> And then you can press `ctl+c` to exit.

### Serverless in an instant

Ok, now that we have serverless installed, let's move through a quick demo.  Let's say that we want to set up a lambda function and an S3 bucket.  We can do so by placing the following in a `serverless.yml` file.  

> You can see this file in the `1-lambda-fn` folder.

```yaml
service: restaurants-app
provider:
  name: aws
  stage: project
  region: us-east-1
functions:
  txqueryfn:
      handler: main.lambda_handler
resources:
  Resources:
    S3Assets:
      Type: AWS::S3::Bucket
      Properties:
        BucketName: txrestaurantreceipts
```

The above creates a new lambda function called `txqueryfn` and also creates a bucket called `txrestaurantreceipts`.

> **Remember** you'll have to set a unique bucket name.

We can create these services in our AWS account with something like the following.

<img src="./deploy-service.png" width="70%">

Ok, let's take a look.

If we go to the s3 we'll see our bucket created.

> <img src="./tx-restaurant-receipts.png" width="40%">

And we'll also see that we created our lambda function.

> <img src="./restaurants-app-tx-query.png" width="50%">

We'll also see that it uploaded the code that we have locally in the `main.py` file.

<img src="./lambda-code.png" width="70%">

Now remove all of the services in our project -- we can quickly do so with the following:

```bash
sls remove
```


### Why Infrastructure as Code

So perhaps by now you can get some of the benefits of infrastructure as code.

With infrastructure as code, we can:
* Explore our cloud architecture in a more readable fashion, by looking at serverless.yml file.
* Reproduce this architecture across different accounts or regions (remember we are doing this in us-east-1, but what if we wanted to reproduce this in a different region).
* See changes to our infrastructure over time (by including our serverless.yml file in our git repository and seeing the git commits associated with that file)
* More easily debug our infrastructure (as all of the set up is stored in a single file)
* Remove all services associated with a project (with the `sls remove` command)

Not bad at all.

### Reading our yaml file

Ok, now let's take another look at our yaml file, and try to understand what's going on.

```yaml
service: restaurants-app
provider:
  name: aws
  stage: project
  runtime: python3.9
  region: us-east-1
functions:
  txqueryfn:
      handler: main.lambda_handler
resources:
  Resources:
    S3Assets:
      Type: AWS::S3::Bucket
      Properties:
        BucketName: txrestaurantreceipts
```

The yaml file above should be fairly readable.

The service is name of our application (just no underscores), and under provider we specify aws and our region.  The `stage` is an AWS feature, that allows us to deploy an architecture for say testing, and production.

And functions are where we list our lambda function.  By specifying our handler, serverless will look for a `main.py` file that has a function called lambda handler.

### Adding a trigger

Ok, now let's update our `serverless.yml` file so that it's triggered by an upload to the s3 bucket.  You can see this code in `2-lambda-fn-trigger`.

```yaml
service: restaurants-app
provider:
  name: aws
  runtime: python3.9
  stage: project
  region: us-east-1
  iam:
    role:
      statements:
        - Effect: Allow
          Action:
            - 's3:GetObject'
          Resource:
            - 'arn:aws:s3:::txrestaurantreceipts/*'
functions:
  getreceipts:
    handler: main.lambda_handler
    events:
      - s3:
          bucket: txrestaurantreceipts
          event: s3:ObjectCreated:*
```

Ok, so there are two changes here. 

* First, at the bottom you can see that under the function we have an `events` listed, which specifies our lambda function should be triggered when s3 object is created.

* Second is that we set a permission to allow our lambda function to read objects.  We specify the permission under the `provider` section, and the permissions will be granted to all functions in the yml file.

> **Notice** that this time we **do not** have a `resources` key.  This is because, the bucket will automatically be created because it's listed under `events`.  

If you go to `2-lambda-fn-trigger`, you'll see the corresponding code.  And if you cd into that folder and type `sls deploy` you'll see services deployed to your aws account (just change the bucket names).

> <img src="./updated-trigger.png" width="60%">

> And importantly, if you click on the role associated with the lambda, you can see that adding our Get object permission added the permission to our lambda function.

<img src="./specify-permissions.png">

### Serverless and cloudformation

If you look closely at any of the folders that has a serverless.yml file, you'll also see a `.serverless` folder.  And if you look inside of there, you'll see various cloudformation files.

<img src="./cloudformation.png">

Cloudformation is AWS's infrastructure as code tool.  And being an IAC tool, it allows us to describe our cloud stack as code, in those json files.

A lot of companies use cloudformation, but it's just more complicated to work with.  So we write a serverless.yml file, and when we call `sls deploy` it uses that file to produce the cloudformation files which are sent to AWS to update our stack.

### IAC vs Boto

So how does an IAC tool like cloudformation or serverless compare to boto?  Here's how we'll divide the responsibilities.

* We'll use serverless to set up the infrastructure -- for example create a lambda function, or an S3 bucket, or setup permissions.
* We'll use boto to execute code *with* this infrastructure.  For example, we can use boto to connect to athena and query our s3 instance.

To use an analogy, reading a `serverless.yml` is essentially like reading a blueprint for a house (or a campus, considering all that's involved).  Calling `sls deploy` updates or builds this campus.  And then when we have code like `boto.client('athena')`, these are the different individuals accomplishing tasks on our campus.  


### Resources

[Serverless](https://aws.amazon.com/blogs/opensource/simplify-development-using-aws-lambda-container-image-with-a-serverless-framework/)

[Serverless AWS S3 trigger](https://www.serverless.com/framework/docs/providers/aws/events/s3)

[Serverless vs Terraform](https://openupthecloud.com/serverless-approaches-comparison/)